In [1]:
import os
import pandas as pd
from jinja2 import Template

# List of years
years = ["2020", "2021", "2022"]  # Add more years as needed

# Base directory containing all the data
base_dir = "C:\\Users\\noahb\\Desktop\\GithubPagesFiles\\Pitching_Pages"

# HTML template for the page
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Pitching Page for {{ year }}</title>
    <style>
        body {
            font-family: Arial, sans-serif;
        }
        .image-container {
            width: 60%;
            margin: 0 auto;
            text-align: center;
            margin-bottom: 35px;
        }
        .image-container img {
            width: 100%;
            max-width: 100%;
        }
        .home-button {
            position: fixed;
            top: 10px;
            right: 10px;
            background-color: #4CAF50;
            color: white;
            padding: 10px 20px;
            text-decoration: none;
            border-radius: 5px;
        }
        .home-button:hover {
            background-color: #45a049;
        }
        h2, h3, h4 {
            text-align: center;
        }
    </style>
</head>
<body>
    <a href="/index.html" class="home-button">Main Page</a>
    <h1>Pitching Overview for {{ year }}</h1>
    <div>
"""

# Function to load and process CSV data
def load_and_process_csv_data(year, base_dir):
    csv_path = os.path.join(base_dir, year, 'Pitching_Overview', 'information.csv')
    df = pd.read_csv(csv_path)
    
    # Filter and sort pitch types by the number of pitches for stand 'ALL'
    df_all_stand = df[df['stand'] == 'ALL'].sort_values(by='Number of Pitches', ascending=False)
    
    pitch_data = {}
    for _, row in df_all_stand.iterrows():
        pitch_type = row['Pitch Type']
        num_pitches_all = row['Number of Pitches']
        
        stands_data = df[df['Pitch Type'] == pitch_type]
        pitch_data[pitch_type] = {'ALL': num_pitches_all, 'stands': {}}
        
        for _, stand_row in stands_data.iterrows():
            stand = stand_row['stand']
            num_pitches = stand_row['Number of Pitches']
            pitch_data[pitch_type]['stands'][stand] = num_pitches
    
    return pitch_data

# Function to generate HTML for a specific year
def generate_html_for_year(year, base_dir):
    pitch_data = load_and_process_csv_data(year, base_dir)

    # Generate the content
    html_content = Template(html_template).render(year=year)
    
    for pitch_type, data in pitch_data.items():
        html_content += f'<h2>{pitch_type} - {data["ALL"]} pitches</h2>'
        for stand, num_pitches in data['stands'].items():
            if stand == 'ALL':
                continue
            html_content += f'<h3>{stand} - {num_pitches} pitches</h3>'
            for i in range(3):  # Assuming there are always 3 plots per stand
                plot_file = f'{pitch_type}_{stand}_Plot_{i}.png'
                plot_path = os.path.join(year, 'Pitching_Overview', plot_file)  # Relative path to plot
                html_content += f'<div class="image-container"><img src="{plot_path}" alt="{plot_file}"></div>'
                html_content += f'<p>Number of pitches: {num_pitches}</p>'
                html_content += '<hr>'  # Horizontal line between plots for separation

    html_content += """
    </div>
</body>
</html>
"""

    # Save the HTML content to a file
    output_dir = os.path.join(base_dir, year, 'Pitching_Overview')
    os.makedirs(output_dir, exist_ok=True)
    html_file_path = os.path.join(output_dir, 'index.html')
    
    with open(html_file_path, 'w') as html_file:
        html_file.write(html_content)
    
    print(f"HTML file created: {html_file_path}")

# Generate HTML files for all years
for year in years:
    generate_html_for_year(year, base_dir)
